In [ ]:
#Waste Type Detector
!pip install -q torchvision
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18
from torch.utils.data import DataLoader, random_split
from PIL import Image
import matplotlib.pyplot as plt
import os
import shutil
import zipfile
from google.colab import files
print("Please upload the TrashNet ZIP file (e.g., archive.zip)")
uploaded = files.upload()
for file in uploaded.keys():
    dataset_zip = file
    break

with zipfile.ZipFile(dataset_zip, 'r') as zip_ref:
    zip_ref.extractall('data')
import glob
search_path = 'data/**/cardboard'
class_folders = glob.glob(search_path, recursive=True)
if class_folders:
    base_path = os.path.dirname(class_folders[0])
else:
    raise Exception("Could not locate class folders (cardboard, glass, etc.)")
bin_path = 'binary_data'
os.makedirs(bin_path + '/recyclable', exist_ok=True)
os.makedirs(bin_path + '/non_recyclable', exist_ok=True)

recyclable_classes = ['cardboard', 'glass', 'metal', 'paper', 'plastic']
non_recyclable_classes = ['trash']

for cls in os.listdir(base_path):
    cls_path = os.path.join(base_path, cls)
    if os.path.isdir(cls_path):
        if cls.lower() in recyclable_classes:
            target_path = os.path.join(bin_path, 'recyclable')
        else:
            target_path = os.path.join(bin_path, 'non_recyclable')
        for img in os.listdir(cls_path):
            img_path = os.path.join(cls_path, img)
            if os.path.isfile(img_path):
                shutil.copy(img_path, target_path)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])
dataset = ImageFolder(root=bin_path, transform=transform)
class_names = dataset.classes
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=16)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Linear(model.fc.in_features, 2)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)
print("Training the model")
for epoch in range(3):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f"Epoch {epoch+1} Loss: {running_loss:.4f}")
print("Upload an image to classify (JPG/PNG)...")
test_upload = files.upload()
def predict_image(img_path, model, transform):
    model.eval()
    image = Image.open(img_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(image_tensor)
        _, predicted = torch.max(outputs, 1)

    return class_names[predicted.item()]
test_img_path = list(test_upload.keys())[0]
predicted_label = predict_image(test_img_path, model, transform)
print(f"Prediction: {predicted_label.upper()}")

if predicted_label == 'recyclable':
    print("Recommendation: Use the Recyclable Bin (Green/Blue)")
else:
    print("Recommendation: Use the Non-Recyclable Bin (Black/Red)")
img = Image.open(test_img_path)
plt.imshow(img)
plt.axis('off')
plt.title(f"Prediction: {predicted_label.upper()}")
plt.show()
